# New bussines location indicator

### Coursera project capstone

This is the course captstone project for the coursera IBM machine learning specialization

### About the project

Having a good location for your business is an important factor for it's prosperity. although the hot spots in a city for a new venue are common knowledge, usually this places have a very high price given that this knowledge is correlated with higher demand. With some datamining and machine learning, some places cam be discovered as good opportunity’s that does not yet received peoples attention.

This project uses python, foursquare api, machine learning and visualization tools to give some insight on where you should open your business in a city.
It works by ranking the places in city by how hot they are for business and them finding what is the optimal place for a new business that does not collide with other similar business by location but also is on a "hot spot" in the city.

The data will on the amount of venues by location will be collected using the foursquare api for a given city. The app/notebook will be created in a generic way so that people using it can change the city name. 
A map of the city based on the amount of venues will be created and also another one for the similar venues as the person is looking to create.

A score for a given location that will be created by adding value for number of venues using mahalanobis distance as a weight and the using the opposite (decreasing score) by having similar venues.
In the and, points will be clustered together to have useful locations as a suggestion for the new business to open.

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen